In [3]:
import numpy as np
from dataclasses import dataclass


##### Process 1

In [4]:
#class for the process 1

@dataclass
class Process1:
    @dataclass
    class State:
        price: int
    level_param: int
    alpha1: float= 0.25

    def up_prob(self,state:State)->float:
        return 1./ (1+ np.exp(-self.alpha1*(self.level_param-state.price)))

    def next_state(self,state:State)->State:
        up_move:int= np.random.binomial(1,self.up_prob(state),)[0]
        return Process1.State(price=state.price+up_move*2-1)


In [5]:
#Generator function to generate sampling traces

def simulation(process, start_state):
    state= start_state
    while True:
        yield state
        state= process.next_state(state)


In [6]:
# Generate num_traces(no. of samples) over time_steps (no. of time steps)

import itertools

def process1_price_traces(
    start_price: int,
    level_param: int,
    alpha1: float,
    time_steps: int,
    num_traces: int
) -> np.ndarray:
    process= Process1(level_param=level_param, alpha1=alpha1)
    start_state= Process1.Sate(price=start_price)
    return np.vstack([
        np.fromiter((s.price for s in itertools.islice(
            simulation(process, start_state),time_steps +1 )), float) 
        for _ in range(num_traces)])


##### Process 2

In [7]:
# mapping function to map the boolean output to int output 

from typing import Mapping, Optional
handy_map: Mapping[Optional[bool],int]={True:-1,False:1, None:0}


In [8]:
#code 

@dataclass
class Process2:   #Abstract class 
    @dataclass
    class State:  #Nested abstract class
        price:int
        is_prev_move_up: Optional[bool]
    alpha2: float= 0.75

    def up_prob(self,state:State)-> float:
        return 0.5*(1+self.alpha2* handy_map[state.is_prev_move_up])
    def next_state(self,state:State)-> State:
        up_move: int= np.random.binomial(1,self.up_prob(state),1)[0]
        return Process2.State(
            price=state.price+up_move*2-1,
            is_prev_move_up=bool(up_move)
        )


In [9]:
#generation of sampling traces of the stock price 

def process2_price_traces(
    start_price:int,
    alpha2: float,
    time_steps:int,
    num_traces: int,
)-> np.ndarray:
    process= Process2(alpha2=alpha2)
    start_state= Process2.State(price=start_price, is_prev_move_up=None)
    return np.vstack([
        np.fromiter((s.price for s in itertools.islice(
            simultation(process,start_state),
            time_steps+1
        )),float) for _ in range(num_traces)
    ])


##### Process 3

In [10]:
# The probability of a up move or a down move dataclass for process3
@dataclass
class Process3:
    @dataclass
    class State:
        num_up_moves: int #attribute of the dataclass State
        num_down_moves: int #attribute of the dataclass Process3
    alpha3: float=1.0

    def up_prob(self,state:State) ->float:
        total= state.sum_up_moves+state.num_down_moves
        if total==0:
            return 0.5
        elif state.num_down_moves==0:
            return state.num_down_moves** self.alpha3
        else:
            return 1./(1+(total/state.num_down_moves-1)**self.alpha3)
        
    def next_state(self,state:State) ->State:
        up_move: int= np.random.binomial(1,self.up_prob(state),1)[0]
        return Process3.State(
            num_up_moves=state.num_up_moves+up_move,
            num_down_moves=state.num_down_moves+1-up_move
        )


In [11]:
#generating sampling traces for the process 3

def process3_price_traces(
        start_price: int,
        alpha3: float,
        time_steps:int,
        num_traces: int,
) ->np.ndarray:
    process= Process3(alpha3=alpha3)
    start_state= Process3.State(num_up_moves=0, num_down_moves=0)
    return np.vstack([
        np.fromiter((start_price.num_up_moves-s.num_down_moves 
                     for s in itertools.isslice(simulation(process,start_state),
                                                time_steps+1)),float)
        for _ in range(num_traces)
    ])


#### 3.3.3. Markov Process Implementation 


In [12]:
from abc import ABC
from dataclasses import dataclass
from typing import Generic, Callable, TypeVar

S= TypeVar('S')
X= TypeVar('X')

class State(ABC, Generic[S]):
    state: S
    def on_non_terminal(
        self,
        f: Callable[['NonTerminal[S]'],X],
        default:X
    ) -> X:
        if isinstance(self,NonTerminal):
            return f(self)
        else: 
            return default

@dataclass(frozen=True)
class Terminal(State[S]):
    state:S
@dataclass(frozen=True)
class NonTerminal(State[S]):
    state:S


In [13]:
import sys 
print(sys.path)


['/opt/anaconda3/envs/cme241/lib/python311.zip', '/opt/anaconda3/envs/cme241/lib/python3.11', '/opt/anaconda3/envs/cme241/lib/python3.11/lib-dynload', '', '/opt/anaconda3/envs/cme241/lib/python3.11/site-packages', '/opt/anaconda3/envs/cme241/lib/python3.11/site-packages/setuptools/_vendor']


In [14]:
#  Markov Class
#  To represent the Markov Process

from abc import abstractmethod
from distribution import Distribution
from typing import Iterable

class MarkovProcess(ABC, Generic[S]):
    @abstractmethod 
    def transition(self, state:NonTerminal[S]) ->Distribution[State[S]]:
        pass
        def simulate(
                self,
                start_state_distribution: Distribution[NonTerminal[S]]
        ) -> Iterable[State[S]]:
            state:State[S]= start_state_distribution.sample()
            yield state
            while isinstance(state, NonTerminal):
                state= self.transituon(state).sample()
                yield state


#### 3.4. Stock Price Examples Modeled as Markov Processes 

##### For Process 3

In [15]:
from distribution import Categorical
from gen_utils.common_funcs import get_unit_sigmoid_func

@dataclass
class StateMP3:
    num_up_moves: int
    num_down_moves: int
@dataclass
class StockPriceMP3(MarkovProcess[StateMP3]):
    alphas3:float= 1.0

    def up_prob(self,state: StateMP3) -> float:
        total= state.num_down_moves+state.num_down_moves
        return get_unit_sigmoid_func(self.alpha3)(
            state.num_doen_moves/total
        ) if total else 0.5
    def transition(
            self,
            state:NonTerminal[StateMP3]
    ) -> Categorical[State[StateMP3]]:
        up_p= self.u_prob(state.state)
        return Categorical({
            NonTerminal(StateMP3(
                state.state.num_up_moves1, state.state.num_down_moves
            )): up_p
            
        })


In [16]:
# Generating sampling traces

from distribution import Constant 
import numpy as np

def process3_price_traces(
        start_price: int,
        alpha3: float,
        time_steps: int,
        num_traces: int
)-> np.ndarray:
    mp= StockPriceMP3(alpha3=alpha3)
    start_state_distribution= Constant(
        NonTerminal(StateMP3(num_up_moves=0, num_down_moves=0))
    )
    return np.vstack([np.fromiter(
        (start_price+s.state.num_up_moves- s.state.num_down_moves for s in 
         itertools.islice(
             mp.simulate(start_state_distribution),
             time_steps+1
         )),
        float
    )for _ in range(num_traces)])


##### For Process 1

##### For Process 2

#### Finite Markov Processes

Markov Process with a finite state space S

In [17]:
# Finite Markov Process dataclass

from typing import Sequence, Mapping
from distribution import FiniteDistribution, Categorical

Transition= Mapping[NonTerminal[S], FiniteDistribution[State[S]]]

class FiniteMarkovProcess(MarkovProcess[S]):
    non_terminal_sates:Sequence[NonTerminal[S]]
    transition_map: Transition[S]

    def __init__(self, transition_map:Mapping[S, FiniteDistribution[S]]):
        non_terminals: Set[S]= set(transition_map.keys())
        self.transition_map={
            NonTerminal(s): Categorical(
                {(NonTerminal(s1) if s1 in non_terminals else Terminal(s1)): p 
                for s1, p in v }
            )
        }
        self.non_terminal_states= list(self.transition_map.keys())
    def __repr__ (self)-> str:
        disply=""
        for s, d in self.transition_map.items():
            display+= f"From State {s.state}:\n"
            for s1, p in d:
                opt= (
                    "Terminal State" if isinstance(s1, Terminal) else "State"
                )
                display+= f"To {opt}{s1.state} with Probability {p:.3f}\n"
            return display
        def transition(self, state: NonTerminal[S]) -> FiniteDistribution[State[S]]:
            return self.tarnsition_map[state]


#### 3.6 Simple Inventory Problem

In [ ]:
from dataclasses import dataclass
from typing import Mapping, Dict
from distribution import Categorical, FiniteDistribution
from markov_process import FiniteMarkovProcess
from scipy.stats import poisson

@dataclass(frozen=True)
class InventoryState:
    on_hand:int
    on_order: int

    def inventory_position(self)-> int:
        return self.on_hand+self.on_order
class SimpleInventoryMPFinite(FiniteMarkovProcess[InventoryState]):
    def __init__(
            self,
            capacity:int,
            poisson_lambda: float
    ):
        self.capacity: int=capacity
        self.oisson_lambda: float= poisson_lambda
        self.poisson_distr= poisson(poisson_lambda)
        super().__init__(self.get_transition_map())

    def get_transition_map(self)-> Mapping[InventoryState, FiniteDistribution[InventoryState]]:
        d: Dict[InventoryState,Categorical[InventoryState]]={}
        for alpha in range(self.capacity+1):
            for beta in range(self.capacity+1-alpha):
                state= InventoryState(alpha,beta)
                ip= state.inventory_position()
                beta1= self.capacity-ip
                state_probs_map:Mapping[InventoryState, float]= {
                    InventoryState(ip-i,beta1):
                    (self.poisson_distr.pmf(i) if i<ip else
                     1-self.poisson_distr.cdf(ip-1))
                    for i in range(ip+1)
                }
                d[InventoryState(alpha,beta)]= Categorical(state_probs_map)
        return d


In [ ]:
#implementation of the Simple Inventory Example 

from dataclasses import dataclass
from typing import Mapping, Dict
from distribution import Categorical, FiniteDistribution
#from markov_process import FiniteMarkovProcess 
from scipy.stats import poisson


if __name__ == '__main__':
    user_capacity= 10
    user_poisson_lambda= 3
    si_mp= SimpleInventoryMPFinite(
        capacity=user_capacity,
        poisson_lambda=user_poisson_lambda
    )
    #print(si_mp)
    print("Transition Map")
    print("--------------")
    print(si_mp)

    print("Stationary Distribution")
    print("-----------------------")
    si_mp.display_stationary_distribution()


Transition Map
--------------
From State InventoryState(on_hand=0, on_order=0):
  To State InventoryState(on_hand=0, on_order=10) with Probability 1.000
From State InventoryState(on_hand=0, on_order=1):
  To State InventoryState(on_hand=1, on_order=9) with Probability 0.050
  To State InventoryState(on_hand=0, on_order=9) with Probability 0.950
From State InventoryState(on_hand=0, on_order=2):
  To State InventoryState(on_hand=2, on_order=8) with Probability 0.050
  To State InventoryState(on_hand=1, on_order=8) with Probability 0.149
  To State InventoryState(on_hand=0, on_order=8) with Probability 0.801
From State InventoryState(on_hand=0, on_order=3):
  To State InventoryState(on_hand=3, on_order=7) with Probability 0.050
  To State InventoryState(on_hand=2, on_order=7) with Probability 0.149
  To State InventoryState(on_hand=1, on_order=7) with Probability 0.224
  To State InventoryState(on_hand=0, on_order=7) with Probability 0.577
From State InventoryState(on_hand=0, on_order=4):